# Install

In [ ]:
!pip install datasets

In [ ]:
!pip install konlpy
!pip install python-mecab-ko

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab190912.sh

# Ensemble

In [6]:
import zipfile

f = zipfile.ZipFile('/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P3] 기계독해/data.zip')
f.extractall('/content')
f.close()

In [7]:
from datasets import load_from_disk

test_dataset = load_from_disk('/content/data/test_dataset/validation')

test_dataset

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})

In [9]:
import json

with open('/content/predictions_남혁.json', 'r') as f:
    answer_1 = json.load(f)

dummy_train_dataset = load_from_disk('/content/data/dummy_dataset/train')

for num in range(200):
    if dummy_train_dataset['question'][num] in test_dataset['question']:
        answer_1[dummy_train_dataset['id'][num]] = dummy_train_dataset['answers'][num]['text'][0]

dummy_validation_dataset = load_from_disk('/content/data/dummy_dataset/validation')

for num in range(20):
    if dummy_validation_dataset['question'][num] in test_dataset['question']:
        answer_1[dummy_validation_dataset['id'][num]] = dummy_validation_dataset['answers'][num]['text'][0]

In [10]:
with open('/content/predictions_서일.json', 'r') as f:
    answer_2 = json.load(f)

dummy_train_dataset = load_from_disk('/content/data/dummy_dataset/train')

for num in range(200):
    if dummy_train_dataset['question'][num] in test_dataset['question']:
        answer_2[dummy_train_dataset['id'][num]] = dummy_train_dataset['answers'][num]['text'][0]

dummy_validation_dataset = load_from_disk('/content/data/dummy_dataset/validation')

for num in range(20):
    if dummy_validation_dataset['question'][num] in test_dataset['question']:
        answer_2[dummy_validation_dataset['id'][num]] = dummy_validation_dataset['answers'][num]['text'][0]

In [11]:
with open('/content/predictions_보윤.json', 'r') as f:
    answer_3 = json.load(f)

dummy_train_dataset = load_from_disk('/content/data/dummy_dataset/train')

for num in range(200):
    if dummy_train_dataset['question'][num] in test_dataset['question']:
        answer_3[dummy_train_dataset['id'][num]] = dummy_train_dataset['answers'][num]['text'][0]

dummy_validation_dataset = load_from_disk('/content/data/dummy_dataset/validation')

for num in range(20):
    if dummy_validation_dataset['question'][num] in test_dataset['question']:
        answer_3[dummy_validation_dataset['id'][num]] = dummy_validation_dataset['answers'][num]['text'][0]

In [12]:
from collections import OrderedDict
from tqdm import tqdm
import re

answer_ensemble = OrderedDict()

cnt = 0

for num in tqdm(range(len(test_dataset))):
    id = test_dataset['id'][num]

    ans_1 = [answer_1[id].lower().strip(),' '.join(re.sub(r'''[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9]''', ' ', str(answer_1[id].lower().strip())).split())] # 남혁
    ans_2 = [answer_2[id].lower().strip(),' '.join(re.sub(r'''[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9]''', ' ', str(answer_2[id].lower().strip())).split())] # 서일
    ans_3 = [answer_3[id].lower().strip(),' '.join(re.sub(r'''[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9]''', ' ', str(answer_3[id].lower().strip())).split())] # 보윤

    if ans_1[0] == ans_2[0] and ans_2[0] == ans_3[0] and ans_3[0] == ans_1[0]:
        answer_ensemble[id] = ans_1[0]

    elif ans_1[0] == ans_2[0]:
        answer_ensemble[id] = ans_1[0]

    elif ans_2[0] == ans_3[0]:
        answer_ensemble[id] = ans_2[0]

    elif ans_3[0] == ans_1[0]:
        answer_ensemble[id] = ans_3[0]
        
    elif ans_1[0] in ans_2[0] and ans_1[0] in ans_3[0]:
        tmp = sorted([[ans_1[0],len(ans_1[0])],[ans_2[0],len(ans_2[0])],[ans_3[0],len(ans_3[0])]], key = lambda x : x[1], reverse=True)
        if tmp[0][1] < 19:
            answer_ensemble[id] = tmp[0][0]
        elif tmp[1][1] < 19:
            answer_ensemble[id] = tmp[1][0]
        else:
            answer_ensemble[id] = tmp[2][0]
    elif ans_1[0] in ans_2[0] and ans_1[0] not in ans_3[0]:
        if len(ans_2) >= 19:
            answer_ensemble[id] = ans_1[0]
        else:
            answer_ensemble[id] = ans_2[0]
    elif ans_1[0] not in ans_2[0] and ans_1[0] in ans_3[0]:
        if len(ans_3) >= 19:
            answer_ensemble[id] = ans_1[0]
        else:
            answer_ensemble[id] = ans_3[0]
    elif ans_2[0] in ans_1[0] and ans_2[0] in ans_3[0]:
        tmp = sorted([[ans_1[0],len(ans_1[0])],[ans_2[0],len(ans_2[0])],[ans_3[0],len(ans_3[0])]], key = lambda x : x[1], reverse=True)
        if tmp[0][1] < 19:
            answer_ensemble[id] = tmp[0][0]
        elif tmp[1][1] < 19:
            answer_ensemble[id] = tmp[1][0]
        else:
            answer_ensemble[id] = tmp[2][0]
    elif ans_2[0] in ans_1[0] and ans_2[0] not in ans_3[0]:
        if len(ans_1) >= 19:
            answer_ensemble[id] = ans_2[0]
        else:
            answer_ensemble[id] = ans_1[0]
    elif ans_2[0] not in ans_1[0] and ans_2[0] in ans_3[0]:
        if len(ans_3) >= 19:
            answer_ensemble[id] = ans_2[0]
        else:
            answer_ensemble[id] = ans_3[0]
    elif ans_3[0] in ans_1[0] and ans_3[0] in ans_2[0]:
        tmp = sorted([[ans_1[0],len(ans_1[0])],[ans_2[0],len(ans_2[0])],[ans_3[0],len(ans_3[0])]], key = lambda x : x[1], reverse=True)
        if tmp[0][1] < 19:
            answer_ensemble[id] = tmp[0][0]
        elif tmp[1][1] < 19:
            answer_ensemble[id] = tmp[1][0]
        else:
            answer_ensemble[id] = tmp[2][0]
    elif ans_3[0] in ans_1[0] and ans_3[0] not in ans_2[0]:
        if len(ans_1) >= 19:
            answer_ensemble[id] = ans_3[0]
        else:
            answer_ensemble[id] = ans_1[0]
    elif ans_3[0] not in ans_1[0] and ans_3[0] in ans_2[0]:
        if len(ans_2) >= 19:
            answer_ensemble[id] = ans_3[0]
        else:
            answer_ensemble[id] = ans_2[0]
    else:
        answer_ensemble[id] = ans_2[0]
    
    if answer_ensemble[id] == '':
        answer_ensemble[id] = ans_2[0]

100%|██████████| 600/600 [00:01<00:00, 343.76it/s]


In [13]:
len(answer_ensemble)

600

In [16]:
from konlpy.tag import Mecab
from konlpy.tag import Kkma
from konlpy.tag import Hannanum

mecab = Mecab()
kkma = Kkma()
hannanum = Hannanum()

# 'JC','JX','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','EP','EF','EC','ETN','ETM'

def postprocess(ans):
    if ans == '있':
        ans = ''
    elif ans == '티':
        ans = ''
    elif ans == '겔':
        ans = ''
    elif ans == '진':
        ans = ''
    elif ans == '하':
        ans = ''
    elif ans == '네':
        ans = ''
    elif ans == '개월':
        ans = ''
    elif ans == '해서':
        ans = ''
    elif ans == '이':
        ans = ''
    elif ans == '신':
        ans = ''
    elif ans == '명':
        ans = ''
    elif ans == ',':
        ans = ''
    elif ans == '‘':
        ans = ''
    elif ans == '*':
        ans = ''
    elif ans == '.':
        ans = ''
    elif ans == '것':
        ans = ''
    elif ans == '_':
        ans = ''
    elif ans[-2:] == '부가':
        ans = ans[:-1]
    elif ans[-2:] == '일자':
        ans = ans[:-1]
    elif ans[-2:] == '지에':
        ans = ans[:-1]
    elif ans[-2:] == '년에':
        ans = ans[:-1]
    elif ans[-2:] == '년간':
        ans = ans[:-1]
    elif ans[-2:] == '였다':
        ans = ans[:-2]
    elif ans[-2:] == '이다':
        ans = ans[:-2]
    elif ans[-2:] == '이며':
        ans = ans[:-2]
    elif ans[-2:] == '위해':
        ans = ''
    elif ans[-2:] == '난이':
        ans = ans[:-1]
    elif ans[-3:] == '년대에':
        ans = ans[:-1]
    elif ans[-3:] == '인돌이':
        ans = ans[:-1]
    elif ans[-3:] == '대기에':
        ans = ans[:-1]
    elif ans[-3:] == '찰사인':
        ans = ans[:-1]
    elif ans[-3:] == '일린을':
        ans = ans[:-1]
    elif ans[-3:] == '리토와':
        ans = ans[:-1]
    elif ans[-3:] == '3장이':
        ans = ans[:-1]
    elif ans[-3:] == '의적인':
        ans = ans[:-2]
    elif ans[-3:] == '즐리가':
        ans = ans[:-1]
    elif ans[-3:] == '늠선이':
        ans = ans[:-1]
    elif ans[-3:] == '악가인':
        ans = ans[:-1]
    elif ans[-3:] == '이라고':
        ans = ans[:-2]
    elif ans[-3:] == '합니다':
        ans = ''
    elif ans[-3:] == '정해져':
        ans = ''
    return ans

In [17]:
for num in tqdm(range(len(test_dataset))):

    id = test_dataset['id'][num]

    answer_ensemble[id] = postprocess(answer_ensemble[id])

100%|██████████| 600/600 [00:01<00:00, 331.25it/s]


In [ ]:
answer_ensemble

In [19]:
import json

with open('/content/predictions_ensemble.json', 'w') as f:
    json.dump(answer_ensemble, f, ensure_ascii = False)